# Intro to the problem

# Import

In [ ]:
# Standard Imports
import os
import pandas as pd
import numpy as np
import sys
import gc
from pathlib import Path
import pickle

# Modelling
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
import lightgbm as ltb
from sklearn.model_selection import (train_test_split, GridSearchCV)


# Visualisation
import plotly.express as px
import plotly.graph_objects as go

# Src modules
sys.path.append(os.path.dirname(os.getcwd())) # Add the parent directory to the Python path so we can import src modules
from src.data_setup import *
from src.model_evaluation import model_eval_pipeline,calc_root_mean_squared_error,calc_root_mean_squared_log_error

#Path Variables
ROOT_PATH =Path(os.path.dirname(os.getcwd()))
DATA_PATH = ROOT_PATH / 'data'
SUBMISSION_PATH = DATA_PATH / 'submissions'
MODEL_PATH=ROOT_PATH/'models/'
DECISIONTREE_PATH=MODEL_PATH/'DecisionTree/'

# Seasonality EDA

# ARIMA

# Prophet

# Decision Trees

Feature Engineering

In [ ]:
# load train,val and test data
train, test, stores, transactions = get_data()
train = clean_train(train)

#Transform train data to Decision Tree Input
df_train =Transform_Data_For_DT(train,60,True)
#Create basic, trend and seasonality features 
df_feats=DT_features(df_train,True)

#Create the split train/validation
df_training,df_validation=train_val_split(df_train)
df_feats_use,df_feats_validation=train_val_split(df_feats)

#Remove Date column for features dataframe
df_feats_validation=df_feats_validation.drop(columns={'date'})
df_feats_use=df_feats_use.drop(columns={'date'})

Modeling

In [ ]:
#Load Model if they exists
#lgbm = load_model(DECISIONTREE_PATH,'lgbm.pkl')
#rf = load_model(DECISIONTREE_PATH,'rf.pkl')
#gbr = load_model(DECISIONTREE_PATH,'gbr.pkl')

#Run each model on the validation data
lgbm_pred=lgbm.predict(df_feats_validation)
rf_pred=rf.predict(df_feats_validation)
gbr_pred=gbr.predict(df_feats_validation)

#Zero all negative values
for i in range(len(lgbm_pred)):
    lgbm_pred[i]=max(0,lgbm_pred[i])
for i in range(len(rf_pred)):
    rf_pred[i]=max(0,rf_pred[i])
for i in range(len(gbr_pred)):
    gbr_pred[i]=max(0,gbr_pred[i])

#Measure RMSLE Score
lgbm_score = calc_root_mean_squared_log_error(df_validation['target'], lgbm_pred)
rf_score = calc_root_mean_squared_log_error(df_validation['target'], rf_pred)
gbr_score = calc_root_mean_squared_log_error(df_validation['target'], gbr_pred)

# Overall Comparison